In [1]:
!python --version

Python 3.10.4


In [29]:
import pickle 
import numpy as np 
import timeit 
import numba 

In [3]:
root = "/home/flavio/datasets/synthetic_layered_graph_1mil/"
layers = []
# nsample = 1000

In [4]:
# with open(root + "fake_" + "education" + "_adjacency_dict.pkl", "rb") as pkl_file:
#     edges = dict(pickle.load(pkl_file))
#     users = list(edges.keys())
#     users = users[:nsample]

In [5]:
layer_types = ["family", "colleague", "education", "neighbor", "household"]
layer_types = ["neighbor", "colleague"]
for ltype in layer_types:
    with open(root + "fake_" + ltype + "_adjacency_dict.pkl", "rb") as pkl_file:
        edges = dict(pickle.load(pkl_file))
        # edges_keep = dict((u, edges[u]) for u in users)
        layers.append(edges)

In [6]:
users = list(layers[0].keys())
len(users)

1000000

In [7]:
print(users[:4])
for user in users: # XXX does this do anything?
    user += 5
print(users[:4])

[0, 1, 2, 3]
[0, 1, 2, 3]


In [8]:
users = [u + 5 for u in users]
print(users[:4])

[5, 6, 7, 8]


In [9]:
node_layer_dict = {}

In [10]:
for user in users:
    node_layer_dict[user] = []
    
    for i, layer in enumerate(layers):
        if user in layer:
            if len(layer[user]) > 0:
                node_layer_dict[user].append(i)

In [46]:
a = list(node_layer_dict.keys())
a[0]
node_layer_dict[5]

[0, 1]

In [43]:
list(layers[0].keys())[:3]
layers[0][3]

[216591,
 359203,
 303294,
 629913,
 941856,
 657939,
 33155,
 184951,
 293721,
 665878,
 214931,
 483588,
 505018,
 838030,
 139399,
 228960,
 629783,
 589567,
 700786,
 428120,
 736350,
 929797,
 462671,
 654239,
 822538,
 973284]

Numba does not work with dicts. Can we replace the following dicts?
- `layers`: a dict (layer type) of dicts (user id) of adjacency lists.
- node_layer_dict: a dict (user id) of layer ids, indicating whether the user has connections in this layer


could we not rewrite this as lists?
- `layers`: one layer is a list of adjacency lists, where the first adjacency list is from the first user
    - the user id refers to the order in the list
    - `layers` is a list of lists of lists 
- `node_layer_dict`: similar to a single layer: the first list indicates which layers are present for the first user

Chatgpt suggests to use numpy arrays for performance. Think; check this response.

In [30]:
# @numba.jit
def custom_sample(choice_set: list):
    "custom function to apply np.random.choice"
    if len(choice_set) == 0:
        return None 
    elif len(choice_set) == 1:
        chosen = choice_set[0]
    else:
        chosen = np.random.choice(choice_set)
    
    return chosen

In [31]:
# @numba.jit
def generate_walks(unique_users: list, walk_len: int, p: float = 0.8):
    "Generate one random walk for each user"
    num_users = len(unique_users)
    random_nums = np.random.rand(num_users, walk_len)

    rows = []
    for user_idx, user in enumerate(unique_users):   
        current_node = user
        
        layer_indices = node_layer_dict[current_node]
        layer_index = custom_sample(layer_indices)
        if not layer_index:
            break

        current_layer = layers[layer_index]
             
        walk = [user]
        while len(walk) < walk_len:
            layer_indices = node_layer_dict[current_node]
        
            roll = random_nums[user_idx][len(walk)]
            
            if roll > p:
                layer_index = custom_sample(layer_indices)
                if not layer:
                    break
                
            adjacent_nodes = current_layer[current_node]

            # Layer index should encode the layer type in an integer 0-4
            walk.append(layer_index)
            
            next_node = custom_sample(adjacent_nodes)
            if not next_node:
                break
            
            walk.append(next_node)
            current_node = next_node
        #assert len(walk) == walk_len, print(len(walk))
        rows.append(walk)

    return rows

    

In [32]:
walk_len = 40
num_walks = 5
a = generate_walks(unique_users=users, walk_len=walk_len)
# call this function num_walks times

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1mUntyped global name 'node_layer_dict':[0m [1m[1mCannot determine Numba type of <class 'dict'>[0m
[1m
File "../../../../../../tmp/ipykernel_133361/220147459.py", line 11:[0m
[1m<source missing, REPL/exec in use?>[0m
[0m

In [25]:
def create_many_walks(users: list, walk_len: int, num_walks: int):
    out = []
    for _ in range(num_walks):
        walks = generate_walks(unique_users=users, walk_len=walk_len)
        out.append(walks)
    return out 

In [28]:
%timeit create_many_walks(users=users, walk_len=walk_len, num_walks=num_walks)

799 ms ± 4.51 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
